In [ ]:
from ms_specific import *

In [ ]:
import constants
if constants.venv_sites_path is not None:
    import site
    site.addsitedir(constants.venv_sites_path)

In [ ]:
import time
import os

# science data packages
import pandas as pd
import numpy as np
import math
import scipy.sparse as ss
import scipy.stats as scipy_stats

# plots related packages
import importlib
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px

In [ ]:
pd.__version__

## loading the data and assigning full names to tickers

In [ ]:
%%time
df = pd.read_pickle('./data/neuroscience_data_yc_2s.pickle')

## subselect tickers with reasonable no. of trades

In [ ]:
import transfer_entropy_evaluator_nbeats_clf as teecr
importlib.reload(teecr)

# compute time can be reduced significantly by using a lighter architecture. 
# We use this setting to show best possible results for TE, relating it to whatb we know about the brain.

te_eval_obj = teecr.TransferEntropyEvaluatorNBeatsClf(
    num_ms_per_timestep=None,
    # decided as per the time lag of information passage in brain     
    time_lag=40,
    num_cores=6,
    negative_sample_for_te=True,
    max_negative_sample_rate=3.0,
    is_hashcode_regularization=True,
    num_hash_funcs=15,
    normalize_te=False,
    is_data_augment_for_te_estimate=True,
    # heavily regualrized     
    augment_multiplier_for_te_estimate=10,
    dir_path_for_saving_hash_models = './saved_hash_models_neuroscience_yc',
)

te, ce = te_eval_obj.te_regression_neuro_multi_trials(
    df=df.copy(),
    num_trials=10,
)

assert te.ndim == 2

te[te < 0] = 0
order_idx = np.array([2, 3, 5, 4, 0])
brain_regions = ['V1', 'LM', 'RL', 'AL', 'AM']

plt.close()
plt.plot(ce[order_idx], 'o', color='crimson')
plt.xticks(range(len(ce)), brain_regions)
plt.ylabel('Conditional Entropy')
plt.show()

fig = px.imshow(
    te[order_idx, :][:, order_idx].T,
    x=brain_regions,
    y=brain_regions,
    text_auto='.1f',    
)

fig.update_layout(
    coloraxis_colorbar_x=0.85,
    margin=dict(l=0,r=0,b=0,t=0),
    font_family="Times New Roman",
    font_size=20,
)

fig.show()